### Installations

In [ ]:
!pip install geopandas
!pip install geopy
!pip install pgeocode
!pip install PyGeodesy
!pip install pyopenssl ndg-httpsclient pyasn1

### Import data

In [6]:
# import libraries
import pandas as pd
import numpy as np
import os
import json

# set path
notebook_path = os.path.abspath('Patient_Clinic_Proximity_Finder.ipynb')
path = notebook_path.rsplit('/',1)[0]

# read data files
patient_data = pd.read_csv(path+'/'+'patients.csv')
clinic_data = pd.read_csv(path+'/'+'clinics.csv')

# read config file
with open('config.json') as config_file:
    cf = json.load(config_file)

In [ ]:
patient_data = patient_data.head(10).copy(deep=True)
clinic_data = clinic_data.head(10).copy(deep=True)

In [ ]:
pat_incomp_entry = {'ID':21,'Address':'random1','Postal Code':'VX1 6L5','FSA':'VX1','City':'Victoria','Province':'BC'}
clin_incomp_entry = {'Clinic ID':21,'Clinic Name':'random1','Clinic Address':'random1','Postal Code':'J4£ 2X1','FSA':'J4£','Clinic City':'Calgary','Province':'AB'}

patient_data = patient_data.append(pat_incomp_entry,ignore_index=True)
clinic_data = clinic_data.append(clin_incomp_entry,ignore_index=True)

In [ ]:
patient_data

In [ ]:
clinic_data

### Map geo co-ordinates

In [7]:
# import libraries
import pgeocode as geo
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

# geopy requirements
locator = Nominatim(user_agent="topcoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

#pgeocode requirements
country = 'ca'
nomi = geo.Nominatim('ca')

def latlon_with_address(ad,cy):
    try:
        r = locator.geocode(ad+' '+cy)
    except (requests.exceptions.ConnectionError,requests.exceptions.Timeout) as e:
        print(f"Error: {e}")
        r = 'No response'
    return r

def latlon_with_street(ad,cy):
    street = re.split('UNIT | SUITE',ad,flags=re.I)[0]
    try:
        r = locator.geocode(street+' '+cy)
    except (requests.exceptions.ConnectionError,requests.exceptions.Timeout) as e:
        print(f"Error: {e}")
        r = 'No response'
    return r

def latlon_with_elim_address(ad,cy):
    add_str = re.split(' |-',ad)
    for i in range(len(add_str)-1):
        sub_str = ' '.join(add_str[:-(i+1)])
        try:
            r = locator.geocode(sub_str+' '+cy)
            if r!=None:
                break
        except (requests.exceptions.ConnectionError,requests.exceptions.Timeout) as e:
            print(f"Error: {e}")
            r = 'No response'
    return r

def latlon_with_fsa(pc):
    try:
        r = nomi.query_postal_code(pc)
    except (requests.exceptions.ConnectionError,requests.exceptions.Timeout) as e:
        print(f"Error: {e}")
        r = 'No response'
    return r

def latlon_with_error_fsa(pc):
    fsa = pc.split(' ')[0]
    all_fsa = list(patient_data['FSA'])+list(clinic_data['FSA'])
    all_fsa.remove(fsa)
    for i in [1,2]:
        for j in all_fsa:
            if fsa[0:len(fsa)-i] in j:
                pc = j
                break
        else:
            continue
        break
    try:
        r = nomi.query_postal_code(pc)
    except (requests.exceptions.ConnectionError,requests.exceptions.Timeout) as e:
        print(f"Error: {e}")
        r = 'No response'
    return r

def lat_long_finder(ad,cy,pc):
    c = 0
    res = latlon_with_address(ad,cy)
    #print('1',type(res))
    if res is None or type(res) is str:
        res = latlon_with_street(ad,cy)
        #print('2',type(res))
        if res is None or type(res) is str:
            res = latlon_with_elim_address(ad,cy)
            #print('3',type(res))
            if res is None or type(res) is str:
                res = latlon_with_fsa(pc)
                #print('4',type(res))
                c+=1
                if res is None or type(res) is str:
                    res = latlon_with_error_fsa(pc)
                    #print('5',type(res))
                    c+=1
    if c==0: return [res.point[0],res.point[1]]
    else: return [res['latitude'],res['longitude']]

# append patient_data
pat_lat = []
pat_lon = []
for index,row in patient_data.iterrows():
    print(f"Fraction completed: {index}/{patient_data.shape[0]+clinic_data.shape[0]}")
    lap = lat_long_finder(row['Address'],row['City'],row['Postal Code'])[0]
    lop = lat_long_finder(row['Address'],row['City'],row['Postal Code'])[1]
    pat_lat.append(lap)
    pat_lon.append(lop)
    
patient_data['Latitude'] = pat_lat
patient_data['Longitude'] = pat_lon
patient_data['Geocode'] = '"'+patient_data['Latitude'].astype(str)+','+patient_data['Longitude'].astype(str)+'"'

# append clinic_data
clin_lat = []
clin_lon = []
for index,row in clinic_data.iterrows():
    print(f"Fraction: {patient_data.shape[0]+index}/{patient_data.shape[0]+clinic_data.shape[0]}")
    lac = lat_long_finder(row['Clinic Address'],row['Clinic City'],row['Postal Code'])[0]
    loc = lat_long_finder(row['Clinic Address'],row['Clinic City'],row['Postal Code'])[1]
    clin_lat.append(lac)
    clin_lon.append(loc)

clinic_data['Latitude'] = clin_lat
clinic_data['Longitude'] = clin_lon
clinic_data['Geocode'] = '"'+clinic_data['Latitude'].astype(str)+','+clinic_data['Longitude'].astype(str)+'"'

Fraction completed: 0/600
Fraction completed: 1/600
Fraction completed: 2/600
Fraction completed: 3/600
Fraction completed: 4/600
Fraction completed: 5/600
Fraction completed: 6/600
Fraction completed: 7/600
Fraction completed: 8/600
Fraction completed: 9/600
Fraction completed: 10/600
Fraction completed: 11/600
Fraction completed: 12/600
Fraction completed: 13/600
Fraction completed: 14/600
Fraction completed: 15/600
Fraction completed: 16/600
Fraction completed: 17/600
Fraction completed: 18/600
Fraction completed: 19/600
Fraction completed: 20/600
Fraction completed: 21/600
Fraction completed: 22/600
Fraction completed: 23/600
Fraction completed: 24/600
Fraction completed: 25/600
Fraction completed: 26/600
Fraction completed: 27/600
Fraction completed: 28/600
Fraction completed: 29/600
Fraction completed: 30/600
Fraction completed: 31/600
Fraction completed: 32/600
Fraction completed: 33/600
Fraction completed: 34/600
Fraction completed: 35/600
Fraction completed: 36/600
Fraction co

Fraction completed: 297/600
Fraction completed: 298/600
Fraction completed: 299/600
Fraction completed: 300/600
Fraction completed: 301/600
Fraction completed: 302/600
Fraction completed: 303/600
Fraction completed: 304/600
Fraction completed: 305/600
Fraction completed: 306/600
Fraction completed: 307/600
Fraction completed: 308/600
Fraction completed: 309/600
Fraction completed: 310/600
Fraction completed: 311/600
Fraction completed: 312/600
Fraction completed: 313/600
Fraction completed: 314/600
Fraction completed: 315/600
Fraction completed: 316/600
Fraction completed: 317/600
Fraction completed: 318/600
Fraction completed: 319/600
Fraction completed: 320/600
Fraction completed: 321/600
Fraction completed: 322/600
Fraction completed: 323/600
Fraction completed: 324/600
Fraction completed: 325/600
Fraction completed: 326/600
Fraction completed: 327/600
Fraction completed: 328/600
Fraction completed: 329/600
Fraction completed: 330/600
Fraction completed: 331/600
Fraction completed: 

In [ ]:
patient_data.head()

In [ ]:
clinic_data.head()

### Calculate distance

In [8]:
def Val2Key(Dict,Val):
    t = list(Dict.values()).count(Val)
    k = []
    if t>1:
        for i in Dict:
            if Dict[i]==Val:
                k.append(i)
        return k
    else:
        return [list(Dict.keys())[list(Dict.values()).index(Val)]] 

In [9]:
# find area around location of interest

from pygeodesy.ellipsoidalVincenty import LatLon 
import requests
import json
import datetime

to = cf['osrm_timeout']
rad = cf['loc_neighbor_dim']
inc = cf['loc_neighbor_expan']
sam = cf['min_clinic_sample_size']

def driving_dist(pat,clin):
    lat1, lon1 = pat[0], pat[1]
    lat2, lon2 = clin[0],clin[1]
    try:
        #raise requests.exceptions.ConnectionError('Throwing and exception here!')
        r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false""",timeout=to)
    except (requests.exceptions.ConnectionError,requests.exceptions.Timeout) as e:
        print(f"Error: {e}")
        r = 'No response'
    if r == 'No response':
        return 10e6
    else:
        try:
            #raise ValueError('this is wrong!')
            routes = json.loads(r.content)
            route_1 = routes.get("routes")
            if route_1 is None:
                route_2 = 10e6
            else:
                route_2 = route_1[0]['distance']
        except ValueError as e:
            print(f"Error: {e}")
            route_2 = 10e6
        return route_2/1000

def neighbour_square(latlon,rad):
    p = LatLon(latlon[0],latlon[1])
    for i in [0,90]:
        deg = i
        d1 = p.destination(rad*1000,deg)
        d2 = p.destination(rad*1000,deg+180)
        if i==0: lat_ran = [d1.lat,d2.lat]
        else: lon_ran = [d1.lon,d2.lon]
    return [lat_ran,lon_ran]

def clinics_in_square(lat_range,lon_range):
    near_clincs = []
    for c in range(clinic_data.shape[0]):
        clin = list(clinic_data.iloc[c][clinic_data.iloc[0].index.isin(['Latitude','Longitude'])])
        if min(lat_range)<=clin[0]<=max(lat_range) and min(lon_range)<=clin[1]<=max(lon_range):
            near_clincs.append(clinic_data.iloc[c]['Clinic ID'])
    return near_clincs

start = datetime.datetime.now()
drive_d = []
for p in range(patient_data.shape[0]):
    print(f"Fraction completed: {p}/{patient_data.shape[0]}")
    #rad = 100
    while True:
        pat = list(patient_data.iloc[p][patient_data.iloc[0].index.isin(['Latitude','Longitude'])])
        lat_range, lon_range = neighbour_square(pat,rad)
        closer_clinics = clinics_in_square(lat_range,lon_range)
        if len(closer_clinics)>=sam: break
        else: rad+=inc

    clin_dist = {}
    for c in closer_clinics:
        clinic = clinic_data[clinic_data['Clinic ID']==c]
        clin = [list(clinic['Latitude'])[0],list(clinic['Longitude'])[0]]
        clin_dist[c] = driving_dist(pat,clin)
    clin_id = Val2Key(clin_dist,min(clin_dist.values()))
    drive_d.append([clin_id[0],min(clin_dist.values())])

print(f"Total runtime: {datetime.datetime.now()-start}")
print(f"Time per patient: {(datetime.datetime.now()-start)/patient_data.shape[0]}")

Clinic_Geoc=clinic_data.set_index('Clinic ID')['Geocode'].to_dict()
Clinic_Add=clinic_data.set_index('Clinic ID')['Clinic Address'].to_dict()
Clinic_PC=clinic_data.set_index('Clinic ID')['Postal Code'].to_dict()
Clinic_FSA=clinic_data.set_index('Clinic ID')['FSA'].to_dict()

clinic_ids = [i[0] for i in drive_d]
clinic_dists = [i[1] for i in drive_d]

Output_dict = {
    'Patient_ID': list(patient_data['ID']),
    'Pat_Geo_Cols': ['"Address,City,Postal Code,FSA"']*patient_data.shape[0],
    'Pat_Geocode': list(patient_data['Geocode']),
    'Pat_Address': list(patient_data['Address']),
    'Pat_Postal_Code': list(patient_data['Postal Code']),
    'Pat_FSA': list(patient_data['FSA']),
    'Nearest_Clinic_ID': clinic_ids,
    'Clinic_Geo_Cols': ['"Clinic Address,Clinic City,Postal Code,FSA"']*patient_data.shape[0],
    'Clinic_Geocode': [Clinic_Geoc[i] for i in clinic_ids],
    'Clinic_Address': [Clinic_Add[i] for i in clinic_ids],
    'Clinic_Postal_Code': [Clinic_PC[i] for i in clinic_ids],
    'Clinic_FSA': [Clinic_FSA[i] for i in clinic_ids],
    'Clinic_Distance': [f'"Clinic ID: {i[0]}, Distance: {i[1]} km"' for i in drive_d]
}


df = pd.DataFrame(Output_dict)

#write outout to file
df.to_csv(path+'/'+'Output.csv',index=False,header=True)

Fraction completed: 0/500
Fraction completed: 1/500
Fraction completed: 2/500
Fraction completed: 3/500
Fraction completed: 4/500
Fraction completed: 5/500
Fraction completed: 6/500
Fraction completed: 7/500
Fraction completed: 8/500
Fraction completed: 9/500
Fraction completed: 10/500
Fraction completed: 11/500
Fraction completed: 12/500
Fraction completed: 13/500
Fraction completed: 14/500
Fraction completed: 15/500
Fraction completed: 16/500
Fraction completed: 17/500
Fraction completed: 18/500
Fraction completed: 19/500
Fraction completed: 20/500
Fraction completed: 21/500
Fraction completed: 22/500
Fraction completed: 23/500
Fraction completed: 24/500
Fraction completed: 25/500
Fraction completed: 26/500
Fraction completed: 27/500
Fraction completed: 28/500
Fraction completed: 29/500
Fraction completed: 30/500
Fraction completed: 31/500
Fraction completed: 32/500
Fraction completed: 33/500
Fraction completed: 34/500
Fraction completed: 35/500
Fraction completed: 36/500
Fraction co

Fraction completed: 297/500
Fraction completed: 298/500
Fraction completed: 299/500
Fraction completed: 300/500
Fraction completed: 301/500
Fraction completed: 302/500
Fraction completed: 303/500
Fraction completed: 304/500
Fraction completed: 305/500
Fraction completed: 306/500
Fraction completed: 307/500
Fraction completed: 308/500
Fraction completed: 309/500
Fraction completed: 310/500
Fraction completed: 311/500
Fraction completed: 312/500
Fraction completed: 313/500
Fraction completed: 314/500
Fraction completed: 315/500
Fraction completed: 316/500
Fraction completed: 317/500
Fraction completed: 318/500
Fraction completed: 319/500
Fraction completed: 320/500
Fraction completed: 321/500
Fraction completed: 322/500
Fraction completed: 323/500
Fraction completed: 324/500
Fraction completed: 325/500
Fraction completed: 326/500
Fraction completed: 327/500
Fraction completed: 328/500
Fraction completed: 329/500
Fraction completed: 330/500
Fraction completed: 331/500
Fraction completed: 

In [ ]:
import json

def driving_dist(pat,clin):
    lat1, lon1 = pat[0], pat[1]
    lat2, lon2 = clin[0],clin[1]
    try:
        #raise requests.exceptions.ConnectionError('Throwing and exception here!')
        r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false""",timeout=to)
    except (requests.exceptions.ConnectionError,requests.exceptions.Timeout) as e:
        print(f"Error: {e}")
        r = 'No response'
    if r == 'No response':
        return 10e6
    else:
        try:
            #raise ValueError('this is wrong!')
            routes = json.loads(r.content)
            route_1 = routes.get("routes")
        except ValueError as e:
            print(f"Error: {e}")
            route_2 = 10e6
        if route_1 is None:
            route_2 = 10e6
        else:
            route_2 = route_1[0]['distance']
        return route_2/1000

drive_d = []
for p in range(0,1):
    print(f"Fraction completed: {p}/{patient_data.shape[0]}")
    #rad = 100
    while True:
        pat = list(patient_data.iloc[p][patient_data.iloc[0].index.isin(['Latitude','Longitude'])])
        lat_range, lon_range = neighbour_square(pat,rad)
        closer_clinics = clinics_in_square(lat_range,lon_range)
        if len(closer_clinics)>=sam: break
        else: rad+=inc

    clin_dist = {}
    for c in closer_clinics:
        clinic = clinic_data[clinic_data['Clinic ID']==c]
        clin = [list(clinic['Latitude'])[0],list(clinic['Longitude'])[0]]
        clin_dist[c] = driving_dist(pat,clin)
    clin_id = Val2Key(clin_dist,min(clin_dist.values()))
    drive_d.append([clin_id[0],min(clin_dist.values())])
    
drive_d

In [ ]:
for i in range(194,195):
    print(i)

In [ ]:
df

### Supplementary

In [ ]:
# import libraries
import pgeocode as geo

#pgeocode requirements
country = 'ca'
nomi = geo.Nominatim('ca')

def lat_long_finder(fsa):
    if np.isnan(nomi.query_postal_code(fsa)['latitude']):
        all_fsa = list(patient_data['FSA'])+list(clinic_data['FSA'])
        all_fsa.remove(fsa)
        for i in [1,2]:
            for j in all_fsa:
                if fsa[0:len(fsa)-i] in j:
                    fsa = j
                    break
            else:
                continue
            break
    FSA_location = nomi.query_postal_code(fsa)
    return [FSA_location['latitude'],FSA_location['longitude']]

# append patient_data
patient_data['Latitude'] = patient_data['FSA'].apply(lambda x: lat_long_finder(x)[0])
patient_data['Longitude'] = patient_data['FSA'].apply(lambda x: lat_long_finder(x)[1])

patient_data['Geocode'] = '('+patient_data['Latitude'].astype(str)+','+patient_data['Longitude'].astype(str)+')'

# append clinic_data
clinic_data['Latitude'] = clinic_data['FSA'].apply(lambda x: lat_long_finder(x)[0])
clinic_data['Longitude'] = clinic_data['FSA'].apply(lambda x: lat_long_finder(x)[1])

clinic_data['Geocode'] = '"'+clinic_data['Latitude'].astype(str)+','+clinic_data['Longitude'].astype(str)+'"'

In [ ]:
# Haversine distance

from sklearn.metrics.pairwise import haversine_distances
from math import radians

def haversine_dist(pat,clin):
    pat_in_radians = [radians(_) for _ in pat]
    clin_in_radians = [radians(_) for _ in clin]
    dist = haversine_distances([pat_in_radians, clin_in_radians])
    dist * 6371000/1000  # multiply by Earth radius to get kilometers
    return dist[0][1]

haver_d = []
for p in range(patient_data.shape[0]):
    clin_dist = {}
    for c in range(clinic_data.shape[0]):
        pat = list(patient_data.iloc[p][patient_data.iloc[0].index.isin(['Latitude','Longitude'])])
        clin = list(clinic_data.iloc[c][clinic_data.iloc[0].index.isin(['Latitude','Longitude'])])
        clin_dist[clinic_data.iloc[c]['Clinic ID']] = haversine_dist(pat,clin)
    haver_d.append(Val2Key(clin_dist,min(clin_dist.values())))

closest_clinic_data = patient_data[['ID']].copy(deep=True)
closest_clinic_data['Haversine_Distance'] = haver_d

In [ ]:
closest_clinic_data.head()

In [ ]:
# driving distance

import requests
import json
import datetime

def driving_dist(pat,clin):
    # call the OSMR API
    lat1, lon1 = pat[0], pat[1]
    lat2, lon2 = clin[0],clin[1]
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon1},{lat1};{lon2},{lat2}?overview=false""")
    # then you load the response using the json libray
    # by default you get only one alternative so you access 0-th element of the `routes`
    routes = json.loads(r.content)
    route_1 = routes.get("routes")[0]
    #print(str(datetime.timedelta(seconds=route_1["duration"])))
    #print(f"Distance in km is: {route_1['distance']/1000}")
    return route_1['distance']/1000

start = datetime.datetime.now()
drive_d = []
for p in range(patient_data.shape[0]):
    clin_dist = {}
    for c in range(clinic_data.shape[0]):
        print(f"Patient: {patient_data.iloc[p]['ID']} Clinic: {clinic_data.iloc[c]['Clinic ID']}")
        pat = list(patient_data.iloc[p][patient_data.iloc[0].index.isin(['Latitude','Longitude'])])
        clin = list(clinic_data.iloc[c][clinic_data.iloc[0].index.isin(['Latitude','Longitude'])])
        clin_dist[clinic_data.iloc[c]['Clinic ID']] = driving_dist(pat,clin)
    drive_d.append(Val2Key(clin_dist,min(clin_dist.values())))

#closest_clinic_data = patient_data[['_ID']].copy(deep=True)
closest_clinic_data['Driving_Distance_BF'] = drive_d

print(f"Total runtime: {datetime.datetime.now()-start}")
print(f"Time per patient: {(datetime.datetime.now()-start)/patient_data.shape[0]}")

In [ ]:
# find area around location of interest

from pygeodesy.ellipsoidalVincenty import LatLon 

def neighbour_square(latlon,rad):
    p = LatLon(latlon[0],latlon[1])
    for i in [0,90]:
        deg = i
        d1 = p.destination(rad*1000,deg)
        d2 = p.destination(rad*1000,deg+180)
        if i==0: lat_ran = [d1.lat,d2.lat]
        else: lon_ran = [d1.lon,d2.lon]
    return [lat_ran,lon_ran]

def clinics_in_square(lat_range,lon_range):
    near_clincs = []
    for c in range(clinic_data.shape[0]):
        clin = list(clinic_data.iloc[c][clinic_data.iloc[0].index.isin(['Latitude','Longitude'])])
        if min(lat_range)<=clin[0]<=max(lat_range) and min(lon_range)<=clin[1]<=max(lon_range):
            near_clincs.append(clinic_data.iloc[c]['Clinic ID'])
    return near_clincs

start = datetime.datetime.now()
drive_d = []
for p in range(patient_data.shape[0]):
    print(f"Fraction completed: {p}/{patient_data.shape[0]}")
    rad = 100
    while True:
        pat = list(patient_data.iloc[p][patient_data.iloc[0].index.isin(['Latitude','Longitude'])])
        lat_range, lon_range = neighbour_square(pat,rad)
        closer_clinics = clinics_in_square(lat_range,lon_range)
        if len(closer_clinics)>=3: break
        else: rad+=50

    clin_dist = {}
    for c in closer_clinics:
        clinic = clinic_data[clinic_data['Clinic ID']==c]
        clin = [list(clinic['Latitude'])[0],list(clinic['Longitude'])[0]]
        clin_dist[c] = driving_dist(pat,clin)
    drive_d.append(Val2Key(clin_dist,min(clin_dist.values())))
    
#closest_clinic_data = patient_data[['ID']].copy(deep=True)
#closest_clinic_data['Driving_Distance_Neighbourhood'] = drive_d

print(f"Total runtime: {datetime.datetime.now()-start}")
print(f"Time per patient: {(datetime.datetime.now()-start)/patient_data.shape[0]}")

In [ ]:
closest_clinic_data['Driving_Distance_BF'].equals(closest_clinic_data['Driving_Distance_Neighbourhood'])